In [1]:
# import libraries
import os
from dotenv import load_dotenv

from langchain_core.tools import StructuredTool

from langchain_chroma import Chroma

from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from pydantic import Field, BaseModel

from typing import Optional

In [2]:
load_dotenv()

True

In [3]:
OPEN_AI_KEY = os.getenv("OPEN_AI_KEY")

In [4]:
# initilizing llm
llm = ChatOpenAI(
    api_key=OPEN_AI_KEY,
    # model="gpt-3.5-turbo",
    model="openai/gpt-4o-mini",
    base_url="https://openrouter.ai/api/v1",
    verbose=True,
)


In [5]:
# connecting to VectorDB

In [6]:
embeddings = OpenAIEmbeddings(
    api_key=OPEN_AI_KEY,
    model="text-embedding-3-large",
    base_url="https://openrouter.ai/api/v1",
)

In [7]:
policy_vectorDB = Chroma(
    embedding_function=embeddings,
    collection_name="Policy",
    persist_directory="Company_Info_VectorDB"
)

In [8]:
previous_record_vectorDB = Chroma(
    collection_name="PreviousData",
    embedding_function=embeddings,
    persist_directory="Company_Info_VectorDB"
)

In [9]:
# tool creation - 1

In [10]:
policy_retriever = policy_vectorDB.similarity_search_with_score("about refund", k= 3)

In [11]:
policy_retriever

[(Document(id='f0f978fc-8872-4a0a-b7b3-bebc4ccf4529', metadata={'priority': 'high', 'source': 'company_policy/refund_policy.txt', 'categories': 'Refund'}, page_content='Refund Policy\n\n1. Refunds are issued only after verifying the order status and payment confirmation.\n2. If an item is damaged, defective, or incorrect, the customer is eligible for either a full refund or a replacement, based on availability.\n3. Refunds are always processed to the original payment method used at checkout.\n4. Once approved, refunds usually reflect within:\n   - 3 to 5 business days for UPI, wallets, and debit/credit cards\n   - 5 to 7 business days for net banking transactions\n5. If a refund is delayed beyond the stated timeline, the ticket must be escalated to the finance team for manual verification.\n\nNon-refundable cases:\n- Items marked as non-returnable\n- Orders cancelled after shipment unless the item is damaged, defective, or incorrect\n'),
  0.9719470143318176),
 (Document(id='f94aa08a-4

In [12]:
# tool creation - 2

In [13]:
# previous_record_retriever = previous_record_vectorDB.as_retriever(search_kwargs={"k": 3})
previous_record_retriever = previous_record_vectorDB.similarity_search_with_score("Payment deducted but order was not placed", k=3)

In [14]:
counter = {}

for doc, confidence in previous_record_retriever:
    counter[doc.page_content]=(doc.metadata['ticket_id'], confidence)
    print()

    
    # print(record[1])
    # print(doc, data)
print(counter)
    #     if type(data) == float:
    #         distance = data
    #         print(f"Confedince: {1 / (1 + distance)}")
    # print()




{'Payment deducted but order was not placed': ('TKT_000001', 0.004158214200288057), 'Order returned but refund not initiated': ('TKT_000013', 0.8301214575767517), 'Order automatically cancelled due to address verification failure': ('TKT_000007', 0.9598504304885864)}


Document(id='4990b4d1-35a6-49dd-9212-a0446712e8c9', metadata={'creation_time': '2024-10-12T09:14:22', 'priority': 'high', 'closure_time': '2024-10-12T13:45:10', 'ticket_id': 'TKT_000001', 'seq_num': 1, 'resolution': 'Payment verified and refund initiated to original payment method within 3–5 business days', 'source': 'previous_record.json', 'category': 'payment'}, page_content='Payment deducted but order was not placed'), 2.450568217682303e-06)

In [15]:
# tool creation - 3

In [16]:
class PreviousRecord(BaseModel):
    source: str

In [17]:
class ResponseDraftingInput(BaseModel):
    ticket_id: str = Field(required=True, description="The Id of ticket you are drafting")
    query: str = Field(required=True, description="Customer issue Text")
    policy: Optional[str] = Field(required=False, description="Relevant policy text to follow strictly")
    previous_record: Optional[tuple] = Field(required=False, description="Similar resolved tickets for reference")

C:\Users\write\AppData\Local\Temp\ipykernel_26872\1093213659.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  ticket_id: str = Field(required=True, description="The Id of ticket you are drafting")
C:\Users\write\AppData\Local\Temp\ipykernel_26872\1093213659.py:3: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  query: str = Field(required=True, description="Customer issue Text")
C:\Users\write\AppData\Local\Temp\ipykernel_26872\1093213659.py:4: PydanticDeprecatedSince20: Using extra keyword argument

In [18]:
class ResponseDraftingOutput(BaseModel):
    ticket_id: str = Field(required=True, description="The Id of ticket you are drafting")
    reply: str = Field(required=True, description="Policy-compliant reply draft for human review")
    tone: str = Field(required=True, description="Tone of response: polite, neutral, apologetic")
    confidence: float = Field(required=True, description="How much You are confident about your draft output")
    used_policy: str = Field(default=None, required=False, description="Policy Reference used in Drafting")
    used_reference_ticket_id: Optional[str] = Field(default=None, required=False, description="id of previous solved Ticket used as reference")

C:\Users\write\AppData\Local\Temp\ipykernel_26872\894788837.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  ticket_id: str = Field(required=True, description="The Id of ticket you are drafting")
C:\Users\write\AppData\Local\Temp\ipykernel_26872\894788837.py:3: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  reply: str = Field(required=True, description="Policy-compliant reply draft for human review")
C:\Users\write\AppData\Local\Temp\ipykernel_26872\894788837.py:4: PydanticDeprecatedSince20: Using

In [19]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

In [20]:
# def response_drafting(data: ResponseDraftingInput) -> ResponseDraftingOutput:
def response_drafting(ticket_id:str, query: str, policy: Optional[str] = None, previous_record: Optional[tuple] = None) -> ResponseDraftingOutput:
    """
    Drafts a safe, policy-compliant customer support response.
    """

    parser = PydanticOutputParser(pydantic_object=ResponseDraftingOutput)
    

    prompt = PromptTemplate(
        
        template="""
    You are a professional customer support agent for a large e-commerce platform.

    Your task:
    - Draft a safe, policy-compliant response for a human support agent to review.
    - Follow the provided policy strictly.
    - Use previous resolved tickets only as reference, not as guarantees.
    - Do NOT make promises outside the policy.
    - Do NOT mention internal processes or timelines unless stated in the policy.
    - Maintain a professional and calm tone at all times.

    Ticket Id:
    {ticket_id}
    
    Customer issue:
    {query}
    
    Relevant policy:
    {policy}
    
    Previous resolved tickets (for reference only):
    {previous_record}
    
    --- OUTPUT RULES ---
    {format_instructions}
    
    If no specific policy applies, set "used_policy" to null.
    """,
    
        input_variables=["ticket_id", "query", "policy", "previous_record"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    

    prompt = prompt.format(
        ticket_id=ticket_id,
        query=query,
        policy=policy or "No specific Policy Provided",
        previous_record=previous_record or "No Previous Records Found",
    )

    llm_result = llm.invoke(prompt)


    structured_output = parser.parse(llm_result.content)

    return structured_output

In [21]:
ResponseDraftingTool = StructuredTool(
    func=response_drafting,
    name="response_drafting",
    description="Generates a safe, policy-based support reply draft",
    verbose=True,
    args_schema=ResponseDraftingInput,
    return_schema=ResponseDraftingOutput
)

In [22]:
# Prepare your test input matching the ResponseDraftingInput schema
test_input = {
    "ticket_id": "TCKT-001",
    "query": "I was double charged for my subscription last month.",
    "policy": "Refund policy: Customers are entitled to a full refund for duplicate charges within 30 days.",
    "previous_record": [{"source": "Ticket #1234: Resolved by issuing credit."}]
}

# Invoke the tool
# This validates the input schema, runs the 'func', and returns the 'return_schema'
result = ResponseDraftingTool.invoke(test_input)



ticket_id='TCKT-001' reply='Dear Customer, \n\nThank you for reaching out to us regarding the double charge on your subscription. We understand how concerning this can be. According to our refund policy, you are entitled to a full refund for duplicate charges made within the last 30 days. Please provide us with the details required to process your refund, and we will assist you further. \n\nThank you for your patience and understanding. \n\nBest regards, \nYour Support Team' tone='polite' confidence=0.9 used_policy='Customers are entitled to a full refund for duplicate charges within 30 days.' used_reference_ticket_id=None

In [23]:
result.confidence

0.9

In [24]:
def get_llm_object(api_key: str, model_name: str='openai/gpt-4o-mini', base_url: str="https://openrouter.ai/api/v1", temperature: float=0.2, verbose: bool=True):
    
    llm = ChatOpenAI(
            model_name=model_name,
            api_key=api_key,
            base_url=base_url,
            temperature=temperature,
            verbose=verbose,
        )
    
    return llm

In [25]:
from langchain_core.output_parsers import StrOutputParser

In [26]:
prompt_template = PromptTemplate(
    template = """
    Your task is to rephase the given text to make it more polite and professional.
    Please ensure that the meaning of the text remains unchanged.
    Text: {current_text}
    """,
    input_variables = ['current_text'],
)

llm = get_llm_object(api_key=OPEN_AI_KEY, model_name='openai/gpt-4o-mini', base_url="https://openrouter.ai/api/v1", temperature=.4, verbose=True)

# prompt = prompt_template.format_prompt(
#     current_text="this mohit kumhar",
# )

parser = StrOutputParser()

rephased_text = prompt_template | llm | parser

result = rephased_text.invoke("this is good product, but service is not good, sometime things take too much time to load , and even support system is also not helpfull")


In [27]:
type(result)

str